In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install keras-metrics

In [0]:
!pip install -U git+https://github.com/qubvel/efficientnet

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-oktxjt25
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-oktxjt25
  Created wheel for efficientnet: filename=efficientnet-1.0.0-cp36-none-any.whl size=17685 sha256=737474f571353662bbe3144c76b009c32d77e728536949d2b89738bbc5c9b414
  Stored in directory: /tmp/pip-ephem-wheel-cache-nws6m_u0/wheels/64/60/2e/30ebaa76ed1626e86bfb0cc0579b737fdb7d9ff8cb9522663a
Successfully built efficientnet


In [0]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, Flatten, Concatenate, Dropout
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from efficientnet.keras import EfficientNetB0, EfficientNetB1
from keras_metrics import recall
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import cv2
import gc
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import random
random.seed(0)
np.random.seed(0)

os.chdir('/content/gdrive/My Drive/Competitions/kaggle/BengaliAI-cv19/nbs')

Using TensorFlow backend.


In [0]:
train = pd.read_csv('/content/gdrive/My Drive/Competitions/kaggle/BengaliAI-cv19/input/train.csv')

In [0]:
train

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো
...,...,...,...,...,...
200835,Train_200835,22,7,2,র্খে
200836,Train_200836,65,9,0,ত্তো
200837,Train_200837,2,1,4,অ্যা
200838,Train_200838,152,9,0,স্নো


In [0]:
grapheme_model = Sequential()
grapheme_model.add(EfficientNetB0(include_top=False, input_shape=(80, 80, 3)))
grapheme_model.add(Flatten())
grapheme_model.add(Dense(168, activation='softmax'))

vow_model = Sequential()
vow_model.add(EfficientNetB0(include_top=False, input_shape=(80, 80, 3)))
vow_model.add(Flatten())
vow_model.add(Dense(11, activation='softmax'))

cons_model = Sequential()
cons_model.add(EfficientNetB0(include_top=False, input_shape=(80, 80, 3)))
cons_model.add(Flatten())
cons_model.add(Dense(7, activation='softmax'))












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
16809984/16804768 [==============================] - 2s 0us/step



In [0]:
cons_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[recall(), 'accuracy'])
grapheme_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[recall(), 'accuracy'])
vow_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[recall(), 'accuracy'])

grapheme_model.metrics_names

['loss', 'recall', 'acc']

In [0]:
g_filepath='/content/gdrive/My Drive/Competitions/kaggle/BengaliAI-cv19/MyCNNV12/grapheme_model'
v_filepath='/content/gdrive/My Drive/Competitions/kaggle/BengaliAI-cv19/MyCNNV12/vow_model'
c_filepath='/content/gdrive/My Drive/Competitions/kaggle/BengaliAI-cv19/MyCNNV12/cons_model'
g_checkpoint = ModelCheckpoint(g_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
v_checkpoint = ModelCheckpoint(v_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
c_checkpoint = ModelCheckpoint(c_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
lrr = ReduceLROnPlateau(monitor='loss', patience=5, verbose=1, factor=0.5, min_lr=0.00001)

g_callbacks_list = [g_checkpoint, lrr]
v_callbacks_list = [v_checkpoint, lrr]
c_callbacks_list = [c_checkpoint, lrr]

In [0]:
model_dict = {
    'cons': cons_model,
    'grapheme': grapheme_model,
    'vow': vow_model
}

callbacks_dict={
    'cons':c_callbacks_list,
    'grapheme':g_callbacks_list,
    'vow':v_callbacks_list
}

In [0]:
epoch = 30
batch_size = 86
weights_paths = {
    'cons':'/content/gdrive/My Drive/Competitions/kaggle/BengaliAI-cv19/MyCNNV12/cons_model_weight.5h',
    'grapheme':'/content/gdrive/My Drive/Competitions/kaggle/BengaliAI-cv19/MyCNNV12/grapheme_model_weight.5h',
    'vow':'/content/gdrive/My Drive/Competitions/kaggle/BengaliAI-cv19/MyCNNV12/vow_model_weight.5h'
}

for i in tqdm(range(4)):
  data = pd.read_parquet(f'/content/gdrive/My Drive/Competitions/kaggle/BengaliAI-cv19/input/train_image_data_{i}.parquet')
  data = data.merge(train, on="image_id").drop('image_id', axis=1)
  images = []
  for k in tqdm(range(data.shape[0])):
    image = cv2.resize((data.loc[k]).values[:-4].reshape(137, 236).astype(float), (120, 160))
    image = (255 - image.reshape(80, 80, 3))>80
    images.append(image.astype(int))
    del image
  images = np.array(images)
  class3 = 7
  target3 = data['consonant_diacritic'].values
  target_classes3 = to_categorical(target3, class3)
  class1 = 168
  target1 = data['grapheme_root'].values
  target_classes1 = to_categorical(target1, class1)
  class2 = 11
  target2 = data['vowel_diacritic'].values
  target_classes2 = to_categorical(target2, class2)
  targets = {
    'cons': target_classes3,
    'grapheme': target_classes1,
    'vow': target_classes2
  }
  del data
  gc.collect()
  train_index, test_index = train_test_split(np.array(range(images.shape[0])), test_size=0.05, random_state=236)
  for j in ['grapheme']:
      print(j)
      data_gen = ImageDataGenerator(
        rotation_range=6,
        zoom_range = 0.15,
        width_shift_range=0.18,
        height_shift_range=0.18)
      data_gen.fit(images[train_index])
      model_dict[j].fit_generator(data_gen.flow(images[train_index], targets[j][train_index], batch_size=batch_size), epochs=epoch,
                                  validation_data = (images[test_index], targets[j][test_index]), callbacks=callbacks_dict[j])
      model_dict[j].save_weights(weights_paths[j])
  del target_classes1, target_classes2, target_classes3, target1, target2, target3, train_index, test_index
  gc.collect()      

grapheme
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/30
555/555 [==============================] - 87s 157ms/step - loss: 3.3085 - recall: 0.1622 - acc: 0.2392 - val_loss: 2.5632 - val_recall: 0.0000e+00 - val_acc: 0.4030

Epoch 00001: val_loss improved from inf to 2.56325, saving model to /content/gdrive/My Drive/Competitions/kaggle/BengaliAI-cv19/MyCNNV12/grapheme_model
Epoch 2/30
457/555 [=======================>......] - ETA: 12s - loss: 1.7796 - recall: 0.3750 - acc: 0.5175